In [74]:
import cv2 as cv
import numpy as np
import shutil
from sklearn.cluster import KMeans
import random
import os

In [75]:
full_set_plader = "Data/KD hele plader"
train_set_plader = "Data/KD train plader"
test_set_plader = "Data/KD Test plader"
train_set_tiles = "Data/KD train tiles" 

train_set_tiles_mixed = train_set_tiles + "/blandet"

crop_percentage = 0.10
circle_size = 0.5

n_clusters=7

In [76]:
def copy_files(files, destination):
    for f in files:
        shutil.copy(os.path.join(full_set_plader, f), os.path.join(destination, f))

In [77]:
def split_data(source_dir, train_set_plader, test_dir, test_size=0.2):
    try:
        # Sikre, at destinationsmapperne eksisterer
        os.makedirs(train_set_plader, exist_ok=True)
        os.makedirs(test_dir, exist_ok=True)

        # Hent alle filnavne fra data-mappen
        all_files = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]

        # Bland listen i state 42 og lav et tilfældigt split
        random.Random(42).shuffle(all_files)
        split_point = int((1 - test_size) * len(all_files))

        # Fordel filerne i train og test mapper
        train_files = all_files[:split_point]
        test_files = all_files[split_point:]

        # Kopier filerne
        copy_files(train_files, train_set_plader)
        copy_files(test_files, test_dir)

        print(f'{len(train_files)} filer blev kopieret til træningsmappen.')
        print(f'{len(test_files)} filer blev kopieret til testmappen.')
    except FileNotFoundError as fnf_error:
        print(f"Fejl: {fnf_error}")
    except Exception as e:
        print(f"En uventet fejl opstod: {e}")

In [78]:
if not os.path.exists(train_set_plader):
    split_data(full_set_plader, train_set_plader, test_set_plader)

59 filer blev kopieret til træningsmappen.
15 filer blev kopieret til testmappen.


In [79]:
def save_tile(tile, output_folder, image_name, x, y):
    # her laver vi en mappe 'blandet' hvis den ikke findes
    if not os.path.exists(os.path.join(output_folder, "blandet")):
        os.makedirs(os.path.join(output_folder, "blandet"))

    # her definerer vi navnet på det tile der skal gemmes som f.eks. 1_3_2.png ved brug af en f-streng
    tile_filename = f"{image_name}_{x}_{y}.png"

    # her definerer bi tile_path som er stedet hvor vi vil gemme vores tile
    tile_path = os.path.join(output_folder, "blandet", tile_filename)
    # her gemmer vi vores tile som tile_filename i folderen 'blandet'
    cv.imwrite(tile_path, tile)

    # her skriver vi til konsollen at vi har gemt vores til i 'blandet' folderen
    print(f"Saved Tile as {tile_filename} in 'blandet' folder")

In [80]:
def get_tiles(image):
    # laver en tom liste 
    tiles = []
    # kører et for loop hvor elementer vil blive tilføjet til listen, hvor y repræsenterer en række af billedet
    for y in range(5):
        tiles.append([])
    # kører et nested loop, hvor billedet bliver delt op i en tavel med 25 kvadrater af 100,100 px og tilføjer til listen tiles
        for x in range(5):
            tiles[-1].append(image[y*100:(y+1)*100, x*100:(x+1)*100])
    return tiles

In [81]:
def zoom_tile(tile):
    height, width = tile.shape[:2]
    
    # Beregner hvor meget der skal beskæres fra hver side
    crop_height = int(height * crop_percentage)
    crop_width = int(width * crop_percentage)
    
    # Opdaterer udsnittets start- og slutpunkter for at fjerne de yderste 5%
    cropped_tile = tile[crop_height:height-crop_height, crop_width:width-crop_width]
    
    return cropped_tile

In [82]:
def remove_circle_from_tile(tile):
    if tile.shape[2] == 3:
        tile = cv.cvtColor(tile, cv.COLOR_BGR2BGRA)
    
    height, width = tile.shape[:2]
    diameter = int(circle_size * min(width, height))
    center = (width // 2, height // 2)
    
    mask = np.zeros((height, width), dtype=np.uint8)
    cv.circle(mask, center, diameter // 2, (255), -1)
    
    # Fjerner cirklen fra alfakanalen, så den bliver gennemsigtig
    tile[mask == 255, 3] = 0

    return tile

In [83]:
def normalize_lighting(tile):
    
    clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img_y_cr_cb = cv.cvtColor(tile, cv.COLOR_BGR2YCrCb)
    img_y_cr_cb[:, :, 0] = clahe.apply(img_y_cr_cb[:, :, 0])

    return cv.cvtColor(img_y_cr_cb, cv.COLOR_YCrCb2BGR)

In [84]:
for filename in os.listdir(train_set_plader):
    image_path = os.path.join(train_set_plader, filename)
    if not os.path.isfile(image_path):
        print(f"Image not found: {image_path}")
        continue
    
    image = cv.imread(image_path)
    tiles = get_tiles(image)
    for y, row in enumerate(tiles):
        for x, tile in enumerate(row):
            tile = zoom_tile(tile)
            tile = remove_circle_from_tile(tile)
            #tile = normalize_lighting(tile)
            save_tile(tile, train_set_tiles, filename, x, y)

Saved Tile as 63.jpg_0_0.png in 'blandet' folder
Saved Tile as 63.jpg_1_0.png in 'blandet' folder
Saved Tile as 63.jpg_2_0.png in 'blandet' folder
Saved Tile as 63.jpg_3_0.png in 'blandet' folder
Saved Tile as 63.jpg_4_0.png in 'blandet' folder
Saved Tile as 63.jpg_0_1.png in 'blandet' folder
Saved Tile as 63.jpg_1_1.png in 'blandet' folder
Saved Tile as 63.jpg_2_1.png in 'blandet' folder
Saved Tile as 63.jpg_3_1.png in 'blandet' folder
Saved Tile as 63.jpg_4_1.png in 'blandet' folder
Saved Tile as 63.jpg_0_2.png in 'blandet' folder
Saved Tile as 63.jpg_1_2.png in 'blandet' folder
Saved Tile as 63.jpg_2_2.png in 'blandet' folder
Saved Tile as 63.jpg_3_2.png in 'blandet' folder
Saved Tile as 63.jpg_4_2.png in 'blandet' folder
Saved Tile as 63.jpg_0_3.png in 'blandet' folder
Saved Tile as 63.jpg_1_3.png in 'blandet' folder
Saved Tile as 63.jpg_2_3.png in 'blandet' folder
Saved Tile as 63.jpg_3_3.png in 'blandet' folder
Saved Tile as 63.jpg_4_3.png in 'blandet' folder
Saved Tile as 63.jpg

In [85]:
def load_images_and_hsv_values(folder):
    images_hsv_values = []
    filenames = []
    for filename in os.listdir(folder):
        img = cv.imread(os.path.join(folder, filename))
        if img is not None and img.any():
            hsv_img = cv.cvtColor(img, cv.COLOR_BGR2HSV)
            hue, saturation, value = np.median(hsv_img, axis=(0,1))
            images_hsv_values.append([hue, saturation, value])
            filenames.append(filename)
    return filenames, np.array(images_hsv_values)

In [86]:
def cluster_images_and_copy(filenames, hsv_values, source_dir):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(hsv_values)
    clusters = kmeans.labels_

    for i in range(n_clusters):
        os.makedirs(f'{train_set_tiles}/category {i}', exist_ok=True)

    for idx, cluster in enumerate(clusters):
        file_path = os.path.join(source_dir, filenames[idx])
        destination_dir = f'{train_set_tiles}/category {cluster}'
        shutil.copy(file_path, destination_dir)

In [87]:
filenames, hsv_values = load_images_and_hsv_values(train_set_tiles_mixed)
cluster_images_and_copy(filenames, hsv_values, train_set_tiles_mixed)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
